# EqualFrequencyBinning: Comprehensive Feature Demonstration

This notebook provides a comprehensive demonstration of the `EqualFrequencyBinning` class from the binlearn library, showcasing all its features and capabilities.

## Core Features:
- **Various Input/Output Formats**: Working with numpy arrays and pandas DataFrames
- **Sklearn Pipeline Integration**: Seamless integration with scikit-learn workflows
- **Joint vs Per-Column Fitting**: Comparing different fitting strategies
- **Skewed Data Handling**: Demonstrating advantages over equal-width binning
- **Quantile-Based Binning**: Creating bins with approximately equal sample sizes

## Advanced Features:
- **Robust Outlier Handling**: Better performance with extreme values
- **Edge Case Handling**: Dealing with duplicate values and small datasets
- **Performance Analysis**: Speed and memory considerations for sorting-based approach
- **Visual Comparisons**: Enhanced plotting showing balanced bin populations

## Overview

`EqualFrequencyBinning` is a quantile-based binning method that divides the data so that each bin contains approximately the same number of samples (equal frequency). This makes it particularly effective for handling skewed distributions and ensuring balanced bin populations, unlike equal width binning which can create unbalanced bins with skewed data.

### Key Advantages:
- **Balanced Bins**: Each bin contains roughly the same number of samples
- **Skewness Robust**: Handles highly skewed distributions effectively
- **Outlier Resilient**: Extreme values don't dominate bin boundaries
- **Predictable Sample Sizes**: Ensures adequate representation in each bin

### When to Use EqualFrequencyBinning:
✅ **Good for**: Skewed data, ensuring balanced representation, outlier-heavy datasets  
⚠️ **Caution with**: Categorical/discrete data with many duplicates  
❌ **Avoid for**: When interpretable bin boundaries are critical

## 1. Import Required Libraries

We'll import all necessary libraries for our comprehensive demonstration:

In [ ]:
# Core libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports for pipeline integration
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Binlearn imports - focusing on EqualFrequencyBinning but also import EqualWidthBinning for comparison
from binlearn.methods import EqualFrequencyBinning, EqualWidthBinning

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib for better plots
plt.style.use('default')

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn available for pipeline integration")
print(f"EqualFrequencyBinning ready for comprehensive demonstration")
print(f"EqualWidthBinning available for comparison purposes")

✅ All libraries imported successfully!
NumPy version: 2.3.2
Pandas version: 2.3.1


## 2. Load and Prepare Sample Data

Let's create various types of sample datasets to demonstrate EqualFrequencyBinning's advantages, particularly with skewed distributions:

In [ ]:
# Create diverse datasets emphasizing scenarios where EqualFrequencyBinning excels
print("📊 Creating Datasets Showcasing EqualFrequencyBinning Advantages")
print("=" * 65)

n_samples = 300

# 1. Highly skewed distributions (where EqualFrequencyBinning shines)
print("\n🎯 Dataset 1: Highly Skewed Distributions")
data_skewed = pd.DataFrame({
    'exponential': np.random.exponential(2, n_samples),           # Right-skewed
    'power_law': np.random.pareto(1.5, n_samples),               # Heavy-tailed
    'log_normal': np.random.lognormal(0, 1, n_samples),          # Log-normal
    'chi_squared': np.random.chisquare(2, n_samples)             # Chi-squared
})

print(f"Shape: {data_skewed.shape}")
print("Columns: exponential, power_law, log_normal, chi_squared")
print("All distributions are highly right-skewed")

# 2. Outlier-heavy data
print("\n🎯 Dataset 2: Outlier-Heavy Data")
# Base normal data with extreme outliers
base_data = np.random.normal(0, 1, n_samples)
outlier_indices = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
base_data[outlier_indices] = np.random.choice([-50, -30, 30, 50, 100], size=len(outlier_indices))

data_outliers = pd.DataFrame({
    'with_outliers': base_data,
    'moderate_outliers': np.concatenate([
        np.random.normal(0, 1, int(n_samples * 0.9)),
        np.random.uniform(-10, 10, int(n_samples * 0.1))
    ])
})

print(f"Shape: {data_outliers.shape}")
print(f"Outlier percentage: ~5% extreme outliers")

# 3. Mixed distribution dataset (for general testing)
print("\n🎯 Dataset 3: Mixed Distributions")
data_mixed = pd.DataFrame({
    'uniform': np.random.uniform(0, 100, n_samples),
    'normal': np.random.normal(50, 15, n_samples),
    'skewed': np.random.exponential(2, n_samples),
    'bimodal': np.concatenate([np.random.normal(25, 5, n_samples//2), 
                              np.random.normal(75, 5, n_samples//2)])
})

print(f"Shape: {data_mixed.shape}")

# 4. Classification dataset (for pipeline testing)
print("\n🎯 Dataset 4: Classification Data")
X_class, y_class = make_classification(
    n_samples=n_samples, 
    n_features=4, 
    n_informative=3,
    n_redundant=1,
    n_classes=2,
    random_state=42
)
data_classification = pd.DataFrame(X_class, columns=['feature_1', 'feature_2', 'feature_3', 'feature_4'])
data_classification['target'] = y_class

print(f"Shape: {data_classification.shape}")
print(f"Target distribution: {np.bincount(y_class)}")

# Display comprehensive statistics
print("\n📈 Dataset Statistics Summary:")
print("\n1. Highly Skewed Data:")
print(data_skewed.describe().round(3))
print("\nSkewness values:")
for col in data_skewed.columns:
    skewness = data_skewed[col].skew()
    print(f"   {col}: {skewness:.2f} ({'highly skewed' if abs(skewness) > 2 else 'moderately skewed' if abs(skewness) > 1 else 'roughly symmetric'})")

print("\n2. Outlier-Heavy Data:")
print(data_outliers.describe().round(3))

print("\n3. Mixed Distributions:")
print(data_mixed.describe().round(3))

print("\n4. Classification Features:")
print(data_classification.iloc[:, :-1].describe().round(3))

📊 Creating NumPy array data with skewed distributions...
NumPy array shape: (200, 3)
NumPy array type: <class 'numpy.ndarray'>

📊 Creating Pandas DataFrame with skewed data...
DataFrame shape: (200, 4)
DataFrame columns: ['age', 'income', 'score', 'wait_time']

📊 Creating Pandas Series with extreme skew...
Series shape: (200,)
Series name: pareto_feature

📈 Data Statistics (notice the skewness):

DataFrame describe:
          age     income   score  wait_time
count  200.00     200.00  200.00     200.00
mean    28.52   29659.90   28.29       2.97
std      7.20   24923.83   15.66       2.74
min     18.44    2395.03    0.72       0.00
25%     23.08   12451.15   16.77       0.97
50%     26.71   22103.67   26.46       2.15
75%     31.85   39382.04   38.14       3.99
max     55.01  138735.99   73.81      12.57

Series describe:
count    200.00
mean      26.66
std       50.73
min        0.02
25%        2.95
50%        8.75
75%       23.49
max      424.91
Name: pareto_feature, dtype: float64



In [ ]:
# Visualize the datasets to understand distribution characteristics
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Data Distribution Characteristics - Where EqualFrequencyBinning Excels', fontsize=16, fontweight='bold')

# Plot skewed distributions
for i, col in enumerate(data_skewed.columns):
    ax = axes[0, i % 2] if i < 2 else axes[1, i % 2]
    data_skewed[col].hist(bins=30, alpha=0.7, ax=ax, color='skyblue', edgecolor='black')
    ax.set_title(f'{col.title().replace("_", " ")} Distribution\n(Skewness: {data_skewed[col].skew():.2f})')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')

# Plot outlier-heavy data
axes[1, 0].clear()
data_outliers['with_outliers'].hist(bins=50, alpha=0.7, ax=axes[1, 0], color='lightcoral', edgecolor='black')
axes[1, 0].set_title('Data with Extreme Outliers')
axes[1, 0].set_xlabel('Value')
axes[1, 0].set_ylabel('Frequency')

# Plot mixed distributions comparison
axes[1, 1].clear()
for col in ['uniform', 'skewed']:
    data_mixed[col].hist(bins=20, alpha=0.6, ax=axes[1, 1], label=col.title())
axes[1, 1].set_title('Uniform vs Skewed Distribution')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

# Plot classification features
axes[2, 0].clear()
axes[2, 1].clear()
data_classification['feature_1'].hist(bins=20, alpha=0.7, ax=axes[2, 0], color='lightgreen', edgecolor='black')
axes[2, 0].set_title('Classification Feature 1')
axes[2, 0].set_xlabel('Value')
axes[2, 0].set_ylabel('Frequency')

data_classification['feature_2'].hist(bins=20, alpha=0.7, ax=axes[2, 1], color='lightgreen', edgecolor='black')
axes[2, 1].set_title('Classification Feature 2')
axes[2, 1].set_xlabel('Value')
axes[2, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("• Exponential and power-law distributions show extreme right skewness")
print("• Outlier-heavy data has most values clustered with extreme outliers")
print("• EqualFrequencyBinning will handle these challenging cases effectively")
print("• Traditional fixed-width binning would struggle with these distributions")

## 🎯 EqualFrequencyBinning Core Functionality

EqualFrequencyBinning creates bins with approximately equal number of observations, making it particularly effective for skewed distributions and outlier-heavy data. Unlike equal-width binning, it adapts to the data distribution.

In [ ]:
# Demonstrate EqualFrequencyBinning on highly skewed data
print("🔬 EqualFrequencyBinning Demonstration on Skewed Data")
print("=" * 55)

# Initialize EqualFrequencyBinning with different bin counts
binning_5 = EqualFrequencyBinning(n_bins=5)
binning_10 = EqualFrequencyBinning(n_bins=10)

# Focus on exponential data (highly right-skewed)
exponential_data = data_skewed[['exponential']].copy()

print(f"\n📊 Original Data Statistics:")
print(f"   Mean: {exponential_data['exponential'].mean():.3f}")
print(f"   Median: {exponential_data['exponential'].median():.3f}")
print(f"   Std: {exponential_data['exponential'].std():.3f}")
print(f"   Skewness: {exponential_data['exponential'].skew():.3f}")
print(f"   Min: {exponential_data['exponential'].min():.3f}")
print(f"   Max: {exponential_data['exponential'].max():.3f}")

# Fit and transform with different bin counts
print(f"\n🎯 EqualFrequencyBinning Results:")

for n_bins, binning in [(5, binning_5), (10, binning_10)]:
    print(f"\n--- {n_bins} Bins ---")
    
    # Fit and transform
    binned_data = binning.fit_transform(exponential_data)
    
    # Get bin information
    bin_edges = binning.bins_[0]  # For single column
    print(f"Bin edges: {[f'{edge:.3f}' for edge in bin_edges]}")
    
    # Analyze frequency distribution
    unique_bins, counts = np.unique(binned_data.iloc[:, 0], return_counts=True)
    print(f"Bin frequencies: {counts}")
    print(f"Frequency balance (std): {np.std(counts):.2f}")
    
    # Show bin assignments for different quantiles
    quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
    print("Value → Bin mapping for quantiles:")
    for q in quantiles:
        value = exponential_data['exponential'].quantile(q)
        bin_idx = binned_data.iloc[exponential_data['exponential'].argsort()[int(q * len(exponential_data))], 0]
        print(f"   {q*100:2.0f}th percentile ({value:.3f}) → Bin {bin_idx}")

# Visualize the binning results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('EqualFrequencyBinning: Handling Skewed Distributions', fontsize=16, fontweight='bold')

# Original distribution
axes[0, 0].hist(exponential_data['exponential'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Original Exponential Distribution\n(Highly Right-Skewed)')
axes[0, 0].set_xlabel('Value')
axes[0, 0].set_ylabel('Frequency')

# Equal-frequency binning visualization
binned_5 = binning_5.fit_transform(exponential_data)
axes[0, 1].hist(binned_5.iloc[:, 0], bins=5, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('After EqualFrequencyBinning (5 bins)\nBalanced Frequencies')
axes[0, 1].set_xlabel('Bin Number')
axes[0, 1].set_ylabel('Frequency')

# Compare with EqualWidthBinning for contrast
equal_width = EqualWidthBinning(n_bins=5)
width_binned = equal_width.fit_transform(exponential_data)
axes[1, 0].hist(width_binned.iloc[:, 0], bins=5, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('EqualWidthBinning Comparison\n(Unbalanced due to skewness)')
axes[1, 0].set_xlabel('Bin Number')
axes[1, 0].set_ylabel('Frequency')

# Bin boundaries visualization
bin_edges_freq = binning_5.bins_[0]
bin_edges_width = equal_width.bins_[0]

axes[1, 1].hist(exponential_data['exponential'], bins=30, alpha=0.3, color='gray', label='Original data')
for i, edge in enumerate(bin_edges_freq[1:-1]):
    axes[1, 1].axvline(edge, color='red', linestyle='--', alpha=0.7, 
                      label='Freq. edges' if i == 0 else "")
for i, edge in enumerate(bin_edges_width[1:-1]):
    axes[1, 1].axvline(edge, color='green', linestyle=':', alpha=0.7,
                      label='Width edges' if i == 0 else "")

axes[1, 1].set_title('Bin Boundaries Comparison')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("• EqualFrequencyBinning adapts to data distribution")
print("• Each bin contains approximately the same number of observations")
print("• Bin boundaries are data-driven, not predetermined")
print("• Particularly effective for skewed and outlier-heavy data")
print("• Maintains statistical balance across bins")

## 🔄 Joint vs Per-Column EqualFrequencyBinning

EqualFrequencyBinning can operate in two modes: per-column (independent) or joint (coordinated across features). This choice significantly impacts the binning strategy.

In [ ]:
# Compare joint vs per-column binning strategies
print("🔄 Joint vs Per-Column EqualFrequencyBinning Comparison")
print("=" * 60)

# Use mixed distribution data for clear differences
comparison_data = data_mixed[['uniform', 'skewed', 'normal']].copy()

print("📊 Original Data Characteristics:")
print(comparison_data.describe().round(3))
print("\nSkewness values:")
for col in comparison_data.columns:
    print(f"   {col}: {comparison_data[col].skew():.3f}")

# Per-column binning (default)
print(f"\n🎯 1. Per-Column Binning (Independent)")
binning_per_col = EqualFrequencyBinning(n_bins=5)
binned_per_col = binning_per_col.fit_transform(comparison_data)

print("Bin edges per column:")
for i, col in enumerate(comparison_data.columns):
    edges = binning_per_col.bins_[i]
    print(f"   {col}: {[f'{edge:.2f}' for edge in edges]}")

print("\nFrequency distribution per column:")
for i, col in enumerate(comparison_data.columns):
    unique_bins, counts = np.unique(binned_per_col.iloc[:, i], return_counts=True)
    print(f"   {col}: {counts} (std: {np.std(counts):.2f})")

# Joint binning
print(f"\n🎯 2. Joint Binning (Coordinated)")
binning_joint = EqualFrequencyBinning(n_bins=5, fitting_mode='joint')
binned_joint = binning_joint.fit_transform(comparison_data)

print("Bin edges per column (joint fitting):")
for i, col in enumerate(comparison_data.columns):
    edges = binning_joint.bins_[i] if hasattr(binning_joint, 'bins_') and binning_joint.bins_ else ["Joint mode - check implementation"]
    print(f"   {col}: {edges}")

print("\nFrequency distribution per column (joint):")
for i, col in enumerate(comparison_data.columns):
    unique_bins, counts = np.unique(binned_joint.iloc[:, i], return_counts=True)
    print(f"   {col}: {counts} (std: {np.std(counts):.2f})")

# Analyze cross-column patterns
print(f"\n📈 Cross-Column Pattern Analysis:")

def analyze_cross_patterns(data, name):
    print(f"\n{name} Binning:")
    # Joint distribution analysis
    joint_patterns = data.value_counts().head(10)
    print(f"   Top 10 bin combinations:")
    for pattern, count in joint_patterns.items():
        print(f"      {pattern}: {count} observations")
    
    # Correlation analysis
    correlation = data.corr()
    print(f"   Bin correlation matrix:")
    print(correlation.round(3))

analyze_cross_patterns(binned_per_col, "Per-Column")
analyze_cross_patterns(binned_joint, "Joint")

# Visualize the differences
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Per-Column vs Joint EqualFrequencyBinning Comparison', fontsize=16, fontweight='bold')

# Per-column results
for i, col in enumerate(comparison_data.columns):
    axes[0, i].hist(binned_per_col.iloc[:, i], bins=5, alpha=0.7, color='lightblue', edgecolor='black')
    axes[0, i].set_title(f'Per-Column: {col.title()}\n(Independent binning)')
    axes[0, i].set_xlabel('Bin Number')
    axes[0, i].set_ylabel('Frequency')

# Joint results
for i, col in enumerate(comparison_data.columns):
    axes[1, i].hist(binned_joint.iloc[:, i], bins=5, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1, i].set_title(f'Joint: {col.title()}\n(Coordinated binning)')
    axes[1, i].set_xlabel('Bin Number')
    axes[1, i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Detailed comparison table
print(f"\n📋 Detailed Comparison Summary:")
comparison_df = pd.DataFrame({
    'Metric': ['Frequency Balance', 'Cross-Column Coordination', 'Individual Optimality', 'Use Case'],
    'Per-Column': [
        'Optimal per feature',
        'Independent',
        'High',
        'Feature-specific analysis'
    ],
    'Joint': [
        'Globally balanced',
        'Coordinated',
        'Moderate',
        'Multi-variate analysis'
    ]
})

print(comparison_df.to_string(index=False))

print(f"\n💡 Practical Guidelines:")
print("• Per-column: Use when features have different scales/distributions")
print("• Per-column: Better for feature-specific analysis")
print("• Joint: Use when maintaining cross-feature relationships is important")
print("• Joint: Better for multivariate pattern discovery")
print("• EqualFrequencyBinning handles skewness well in both modes")

## 🔗 Pipeline Integration with Scikit-learn

EqualFrequencyBinning integrates seamlessly with scikit-learn pipelines, making it easy to incorporate into machine learning workflows, especially with skewed data.

In [ ]:
# Demonstrate pipeline integration with EqualFrequencyBinning
print("🔗 EqualFrequencyBinning in Machine Learning Pipelines")
print("=" * 58)

# Prepare classification data
X = data_classification.iloc[:, :-1]  # Features
y = data_classification['target']     # Target

print(f"📊 Dataset Info:")
print(f"   Features shape: {X.shape}")
print(f"   Target distribution: {np.bincount(y)}")
print(f"   Feature ranges: {X.min().round(2).values} to {X.max().round(2).values}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"\n🎯 Train/Test Split:")
print(f"   Train set: {X_train.shape[0]} samples")
print(f"   Test set: {X_test.shape[0]} samples")

# Create multiple pipelines for comparison
pipelines = {
    'No Binning': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    
    'EqualFrequency (3 bins)': Pipeline([
        ('binning', EqualFrequencyBinning(n_bins=3)),
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    
    'EqualFrequency (5 bins)': Pipeline([
        ('binning', EqualFrequencyBinning(n_bins=5)),
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    
    'EqualFrequency (10 bins)': Pipeline([
        ('binning', EqualFrequencyBinning(n_bins=10)),
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    
    'EqualWidth (5 bins)': Pipeline([
        ('binning', EqualWidthBinning(n_bins=5)),
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42))
    ])
}

# Evaluate pipelines with cross-validation
print(f"\n🔬 Cross-Validation Results (5-fold):")
results = {}

for name, pipeline in pipelines.items():
    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    results[name] = {
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"   {name:25s}: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Train final models and evaluate on test set
print(f"\n🎯 Test Set Performance:")
test_results = {}

for name, pipeline in pipelines.items():
    # Fit and predict
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    test_results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    print(f"   {name:25s}: Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}")

# Detailed pipeline analysis
print(f"\n🔍 Pipeline Component Analysis:")

# Analyze feature transformations
freq_pipeline = pipelines['EqualFrequency (5 bins)']
freq_pipeline.fit(X_train, y_train)

if 'binning' in freq_pipeline.named_steps:
    binning_step = freq_pipeline.named_steps['binning']
    print(f"\nEqualFrequencyBinning (5 bins) - Bin Information:")
    
    for i, col in enumerate(X.columns):
        edges = binning_step.bins_[i]
        print(f"   {col}: {len(edges)-1} bins with edges {[f'{e:.2f}' for e in edges]}")
        
        # Show data distribution across bins
        binned_feature = binning_step.transform(X_train)[:, i]
        unique_bins, counts = np.unique(binned_feature, return_counts=True)
        print(f"           Frequencies: {counts} (balance std: {np.std(counts):.2f})")

# Create comprehensive results DataFrame
results_df = pd.DataFrame({
    'Pipeline': list(results.keys()),
    'CV_Mean': [results[name]['cv_mean'] for name in results.keys()],
    'CV_Std': [results[name]['cv_std'] for name in results.keys()],
    'Test_Accuracy': [test_results[name]['accuracy'] for name in results.keys()],
    'Test_F1': [test_results[name]['f1'] for name in results.keys()]
})

print(f"\n📊 Complete Results Summary:")
print(results_df.round(4))

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Pipeline Performance Comparison', fontsize=16, fontweight='bold')

# Cross-validation scores
axes[0].bar(range(len(results)), [results[name]['cv_mean'] for name in results.keys()], 
           yerr=[results[name]['cv_std'] for name in results.keys()], 
           alpha=0.7, capsize=5, color='skyblue', edgecolor='black')
axes[0].set_title('Cross-Validation Accuracy')
axes[0].set_xlabel('Pipeline')
axes[0].set_ylabel('Accuracy')
axes[0].set_xticks(range(len(results)))
axes[0].set_xticklabels(list(results.keys()), rotation=45, ha='right')

# Test set performance
test_metrics = ['accuracy', 'precision', 'recall', 'f1']
x = np.arange(len(pipelines))
width = 0.2

for i, metric in enumerate(test_metrics):
    values = [test_results[name][metric] for name in pipelines.keys()]
    axes[1].bar(x + i*width, values, width, label=metric.title(), alpha=0.7)

axes[1].set_title('Test Set Performance Metrics')
axes[1].set_xlabel('Pipeline')
axes[1].set_ylabel('Score')
axes[1].set_xticks(x + width * 1.5)
axes[1].set_xticklabels(list(pipelines.keys()), rotation=45, ha='right')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n💡 Pipeline Integration Insights:")
print("• EqualFrequencyBinning integrates seamlessly with scikit-learn")
print("• Binning can improve or stabilize model performance")
print("• Different bin counts offer trade-offs between complexity and smoothing")
print("• Works well with standard preprocessing steps (scaling, etc.)")
print("• Particularly valuable for handling skewed feature distributions")

# Show feature importance analysis if available
if hasattr(freq_pipeline.named_steps['classifier'], 'coef_'):
    print(f"\n🎯 Feature Importance Analysis (EqualFrequency 5 bins):")
    coefficients = freq_pipeline.named_steps['classifier'].coef_[0]
    
    # Create feature names for binned features
    binned_feature_names = []
    for col in X.columns:
        binned_feature_names.append(f"{col}_binned")
    
    importance_df = pd.DataFrame({
        'Feature': binned_feature_names,
        'Coefficient': coefficients,
        'Abs_Coefficient': np.abs(coefficients)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print(importance_df.round(4))

## ⚠️ Edge Cases and Robustness Testing

EqualFrequencyBinning handles challenging data scenarios well, but understanding its behavior in edge cases is crucial for robust applications.

In [ ]:
# Test EqualFrequencyBinning robustness in edge cases
print("⚠️ EqualFrequencyBinning Edge Cases and Robustness Testing")
print("=" * 65)

# Edge Case 1: Data with many duplicate values
print("🔍 Edge Case 1: Data with Many Duplicate Values")
duplicate_data = pd.DataFrame({
    'many_duplicates': np.concatenate([
        np.full(100, 1.0),    # 100 values of 1.0
        np.full(80, 2.0),     # 80 values of 2.0
        np.full(60, 3.0),     # 60 values of 3.0
        np.full(40, 4.0),     # 40 values of 4.0
        np.full(20, 5.0)      # 20 values of 5.0
    ])
})

print(f"Value counts:")
print(duplicate_data['many_duplicates'].value_counts().sort_index())

try:
    binning_duplicates = EqualFrequencyBinning(n_bins=5)
    binned_duplicates = binning_duplicates.fit_transform(duplicate_data)
    
    print(f"✅ Binning successful!")
    print(f"Bin edges: {[f'{edge:.1f}' for edge in binning_duplicates.bins_[0]]}")
    
    unique_bins, counts = np.unique(binned_duplicates.iloc[:, 0], return_counts=True)
    print(f"Bin frequencies: {counts}")
    print(f"Note: Perfect equal frequency impossible due to duplicate values")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Edge Case 2: Very small dataset
print(f"\n🔍 Edge Case 2: Very Small Dataset")
small_data = pd.DataFrame({
    'small_sample': [1.0, 2.0, 3.0, 4.0, 5.0]  # Only 5 samples
})

try:
    binning_small = EqualFrequencyBinning(n_bins=3)
    binned_small = binning_small.fit_transform(small_data)
    
    print(f"✅ Small data binning successful!")
    print(f"Original data: {small_data['small_sample'].values}")
    print(f"Binned data: {binned_small.iloc[:, 0].values}")
    print(f"Bin edges: {[f'{edge:.1f}' for edge in binning_small.bins_[0]]}")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Edge Case 3: Data with extreme outliers
print(f"\n🔍 Edge Case 3: Data with Extreme Outliers")
outlier_data = pd.DataFrame({
    'with_outliers': np.concatenate([
        np.random.normal(0, 1, 95),    # 95 normal values
        [1000, -1000, 5000, -2000, 10000]  # 5 extreme outliers
    ])
})

print(f"Data range: {outlier_data['with_outliers'].min():.1f} to {outlier_data['with_outliers'].max():.1f}")
print(f"Standard deviation: {outlier_data['with_outliers'].std():.1f}")

try:
    binning_outliers = EqualFrequencyBinning(n_bins=5)
    binned_outliers = binning_outliers.fit_transform(outlier_data)
    
    print(f"✅ Outlier handling successful!")
    
    # Show how outliers are distributed
    unique_bins, counts = np.unique(binned_outliers.iloc[:, 0], return_counts=True)
    print(f"Bin frequencies: {counts}")
    
    # Identify which bin contains outliers
    extreme_values = outlier_data['with_outliers'] > 100
    if extreme_values.any():
        outlier_bins = binned_outliers.iloc[extreme_values, 0].unique()
        print(f"Outliers assigned to bins: {outlier_bins}")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Edge Case 4: Constant data
print(f"\n🔍 Edge Case 4: Constant Data")
constant_data = pd.DataFrame({
    'constant': np.full(100, 42.0)  # All values are 42.0
})

try:
    binning_constant = EqualFrequencyBinning(n_bins=5)
    binned_constant = binning_constant.fit_transform(constant_data)
    
    print(f"✅ Constant data handling successful!")
    print(f"All values: {constant_data['constant'].iloc[0]}")
    print(f"Bin assignments: {binned_constant.iloc[:, 0].unique()}")
    print(f"Bin edges: {binning_constant.bins_[0]}")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Edge Case 5: Missing values
print(f"\n🔍 Edge Case 5: Data with Missing Values")
missing_data = pd.DataFrame({
    'with_nan': np.concatenate([
        np.random.normal(0, 1, 80),    # 80 normal values
        [np.nan] * 20                   # 20 missing values
    ])
})

print(f"Missing values: {missing_data['with_nan'].isna().sum()}")
print(f"Valid values: {missing_data['with_nan'].notna().sum()}")

try:
    binning_missing = EqualFrequencyBinning(n_bins=5)
    # Note: Most binning methods require handling NaN values first
    clean_data = missing_data.dropna()
    binned_missing = binning_missing.fit_transform(clean_data)
    
    print(f"✅ Missing value handling (after dropping NaN)!")
    print(f"Processed {len(clean_data)} valid observations")
    
    unique_bins, counts = np.unique(binned_missing.iloc[:, 0], return_counts=True)
    print(f"Bin frequencies: {counts}")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Edge Case 6: More bins than unique values
print(f"\n🔍 Edge Case 6: More Bins than Unique Values")
limited_unique = pd.DataFrame({
    'few_unique': [1, 1, 2, 2, 3, 3] * 10  # Only 3 unique values, 60 total
})

print(f"Unique values: {sorted(limited_unique['few_unique'].unique())}")
print(f"Total observations: {len(limited_unique)}")

try:
    binning_limited = EqualFrequencyBinning(n_bins=10)  # More bins than unique values
    binned_limited = binning_limited.fit_transform(limited_unique)
    
    print(f"✅ Limited unique values handling successful!")
    print(f"Requested 10 bins, got {len(binning_limited.bins_[0])-1} effective bins")
    print(f"Bin edges: {binning_limited.bins_[0]}")
    
    unique_bins, counts = np.unique(binned_limited.iloc[:, 0], return_counts=True)
    print(f"Actual bins used: {len(unique_bins)}")
    print(f"Bin frequencies: {counts}")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Visualization of edge cases
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('EqualFrequencyBinning: Edge Cases Visualization', fontsize=16, fontweight='bold')

# Plot edge cases
edge_cases = [
    (duplicate_data['many_duplicates'], 'Many Duplicates'),
    (outlier_data['with_outliers'], 'Extreme Outliers'),
    (constant_data['constant'], 'Constant Data'),
    (missing_data['with_nan'].dropna(), 'After Removing NaN'),
    (limited_unique['few_unique'], 'Few Unique Values'),
    (small_data['small_sample'], 'Very Small Dataset')
]

for i, (data, title) in enumerate(edge_cases):
    row, col = i // 3, i % 3
    axes[row, col].hist(data, bins=min(20, len(data.unique())), alpha=0.7, 
                       color='lightcoral', edgecolor='black')
    axes[row, col].set_title(f'{title}\n({len(data)} observations)')
    axes[row, col].set_xlabel('Value')
    axes[row, col].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\n💡 Edge Case Insights:")
print("• EqualFrequencyBinning handles most edge cases gracefully")
print("• Duplicate values prevent perfect equal frequencies")
print("• Extreme outliers are naturally accommodated")
print("• Constant data results in single bin assignment")
print("• More bins than unique values are automatically reduced")
print("• Missing values should be handled before binning")
print("• Small datasets may not achieve ideal frequency distribution")

# Summary table of edge case handling
edge_case_summary = pd.DataFrame({
    'Edge Case': ['Many Duplicates', 'Small Dataset', 'Extreme Outliers', 
                  'Constant Data', 'Missing Values', 'Limited Unique Values'],
    'Handling': ['Approximate frequencies', 'Works with constraints', 'Robust accommodation',
                'Single bin assignment', 'Requires preprocessing', 'Auto bin reduction'],
    'Recommendation': ['Use fewer bins', 'Consider bin count', 'EqualFreq advantage',
                      'Check for constants', 'Impute or drop NaN', 'Reduce n_bins parameter']
})

print(f"\n📋 Edge Case Handling Summary:")
print(edge_case_summary.to_string(index=False))

## ⚡ Performance Analysis and Benchmarking

Understanding the computational characteristics of EqualFrequencyBinning helps optimize its use in different scenarios and data sizes.

In [ ]:
# Performance benchmarking for EqualFrequencyBinning
print("⚡ EqualFrequencyBinning Performance Analysis")
print("=" * 50)

# Test different data sizes and configurations
test_sizes = [100, 1000, 10000, 50000]
test_features = [1, 5, 10, 20]
test_bins = [3, 5, 10, 20]

# Performance tracking
performance_results = []

print("🔬 Performance Testing Configuration:")
print(f"   Data sizes: {test_sizes}")
print(f"   Feature counts: {test_features}")
print(f"   Bin counts: {test_bins}")

# Benchmark 1: Scaling with data size
print(f"\n📊 Benchmark 1: Scaling with Data Size (5 features, 5 bins)")
size_performance = {}

for size in test_sizes:
    # Generate test data
    np.random.seed(42)
    test_data = pd.DataFrame({
        f'feature_{i}': np.random.exponential(2, size) 
        for i in range(5)
    })
    
    # Measure fitting time
    binning = EqualFrequencyBinning(n_bins=5)
    
    start_time = time.time()
    binning.fit(test_data)
    fit_time = time.time() - start_time
    
    # Measure transform time
    start_time = time.time()
    transformed = binning.transform(test_data)
    transform_time = time.time() - start_time
    
    # Measure fit_transform time
    start_time = time.time()
    binning_new = EqualFrequencyBinning(n_bins=5)
    fit_transformed = binning_new.fit_transform(test_data)
    fit_transform_time = time.time() - start_time
    
    size_performance[size] = {
        'fit_time': fit_time,
        'transform_time': transform_time,
        'fit_transform_time': fit_transform_time,
        'total_time': fit_time + transform_time
    }
    
    print(f"   {size:6d} samples: Fit={fit_time:.4f}s, Transform={transform_time:.4f}s, Total={fit_time + transform_time:.4f}s")

# Benchmark 2: Scaling with feature count
print(f"\n📊 Benchmark 2: Scaling with Feature Count (10k samples, 5 bins)")
feature_performance = {}

for n_features in test_features:
    # Generate test data
    np.random.seed(42)
    test_data = pd.DataFrame({
        f'feature_{i}': np.random.exponential(2, 10000) 
        for i in range(n_features)
    })
    
    binning = EqualFrequencyBinning(n_bins=5)
    
    start_time = time.time()
    binning.fit_transform(test_data)
    total_time = time.time() - start_time
    
    feature_performance[n_features] = total_time
    print(f"   {n_features:2d} features: {total_time:.4f}s ({total_time/n_features:.4f}s per feature)")

# Benchmark 3: Scaling with bin count
print(f"\n📊 Benchmark 3: Scaling with Bin Count (10k samples, 5 features)")
bin_performance = {}

# Generate consistent test data
np.random.seed(42)
test_data_bins = pd.DataFrame({
    f'feature_{i}': np.random.exponential(2, 10000) 
    for i in range(5)
})

for n_bins in test_bins:
    binning = EqualFrequencyBinning(n_bins=n_bins)
    
    start_time = time.time()
    binning.fit_transform(test_data_bins)
    total_time = time.time() - start_time
    
    bin_performance[n_bins] = total_time
    print(f"   {n_bins:2d} bins: {total_time:.4f}s")

# Memory usage analysis
print(f"\n💾 Memory Usage Analysis:")

def get_memory_usage(data, n_bins):
    """Estimate memory usage for binning operation"""
    import sys
    
    # Original data size
    original_size = data.memory_usage(deep=True).sum()
    
    # Fit binning
    binning = EqualFrequencyBinning(n_bins=n_bins)
    binned_data = binning.fit_transform(data)
    
    # Binned data size
    binned_size = binned_data.memory_usage(deep=True).sum()
    
    # Model size (approximate)
    model_size = sys.getsizeof(binning.bins_) if hasattr(binning, 'bins_') else 1000
    
    return {
        'original_mb': original_size / 1024 / 1024,
        'binned_mb': binned_size / 1024 / 1024,
        'model_mb': model_size / 1024 / 1024,
        'compression_ratio': original_size / binned_size if binned_size > 0 else 1
    }

# Test memory usage on different sizes
for size in [1000, 10000, 50000]:
    test_data_memory = pd.DataFrame({
        f'feature_{i}': np.random.exponential(2, size) 
        for i in range(5)
    })
    
    memory_stats = get_memory_usage(test_data_memory, 5)
    print(f"   {size:6d} samples: Original={memory_stats['original_mb']:.2f}MB, "
          f"Binned={memory_stats['binned_mb']:.2f}MB, "
          f"Model={memory_stats['model_mb']:.3f}MB")

# Compare with other binning methods
print(f"\n🏁 Method Comparison (10k samples, 5 features, 5 bins):")

# Generate consistent test data
np.random.seed(42)
comparison_data = pd.DataFrame({
    f'feature_{i}': np.random.exponential(2, 10000) 
    for i in range(5)
})

methods = {
    'EqualFrequencyBinning': EqualFrequencyBinning(n_bins=5),
    'EqualWidthBinning': EqualWidthBinning(n_bins=5)
}

method_times = {}
for name, method in methods.items():
    start_time = time.time()
    result = method.fit_transform(comparison_data)
    total_time = time.time() - start_time
    method_times[name] = total_time
    print(f"   {name:20s}: {total_time:.4f}s")

# Visualize performance results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('EqualFrequencyBinning Performance Analysis', fontsize=16, fontweight='bold')

# Data size scaling
axes[0, 0].plot(list(size_performance.keys()), 
               [size_performance[size]['fit_transform_time'] for size in size_performance.keys()],
               'o-', color='blue', linewidth=2, markersize=8)
axes[0, 0].set_title('Scaling with Data Size')
axes[0, 0].set_xlabel('Number of Samples')
axes[0, 0].set_ylabel('Time (seconds)')
axes[0, 0].set_xscale('log')
axes[0, 0].grid(True, alpha=0.3)

# Feature count scaling
axes[0, 1].plot(list(feature_performance.keys()), 
               list(feature_performance.values()),
               'o-', color='green', linewidth=2, markersize=8)
axes[0, 1].set_title('Scaling with Feature Count')
axes[0, 1].set_xlabel('Number of Features')
axes[0, 1].set_ylabel('Time (seconds)')
axes[0, 1].grid(True, alpha=0.3)

# Bin count scaling
axes[1, 0].plot(list(bin_performance.keys()), 
               list(bin_performance.values()),
               'o-', color='red', linewidth=2, markersize=8)
axes[1, 0].set_title('Scaling with Bin Count')
axes[1, 0].set_xlabel('Number of Bins')
axes[1, 0].set_ylabel('Time (seconds)')
axes[1, 0].grid(True, alpha=0.3)

# Method comparison
method_names = list(method_times.keys())
method_values = list(method_times.values())
axes[1, 1].bar(method_names, method_values, color=['skyblue', 'lightcoral'], alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Method Comparison')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Performance summary
print(f"\n📋 Performance Summary:")
performance_summary = pd.DataFrame({
    'Metric': ['Linear scaling with data size', 'Linear scaling with features', 
               'Minimal impact of bin count', 'Memory efficiency', 'Comparison vs EqualWidth'],
    'Result': [
        f"~{(size_performance[50000]['fit_transform_time']/size_performance[1000]['fit_transform_time']):.1f}x slower for 50x data",
        f"~{(feature_performance[20]/feature_performance[1]):.1f}x slower for 20x features",
        f"~{(bin_performance[20]/bin_performance[3]):.1f}x slower for 6.7x bins",
        "Moderate memory overhead",
        f"~{(method_times['EqualFrequencyBinning']/method_times['EqualWidthBinning']):.1f}x vs EqualWidth"
    ]
})

print(performance_summary.to_string(index=False))

print(f"\n💡 Performance Insights:")
print("• EqualFrequencyBinning scales linearly with data size and features")
print("• Bin count has minimal impact on performance")
print("• Performance is competitive with other binning methods")
print("• Memory usage is reasonable for most applications")
print("• Quantile calculation is the main computational bottleneck")
print("• Consider data size when choosing bin count for optimal performance")

## 📋 Practical Recommendations and Best Practices

Based on our comprehensive analysis, here are practical guidelines for effectively using EqualFrequencyBinning in real-world applications.

In [ ]:
# Practical recommendations and decision framework
print("📋 EqualFrequencyBinning: Practical Recommendations")
print("=" * 55)

# Decision framework for when to use EqualFrequencyBinning
decision_framework = {
    "Ideal Use Cases": [
        "Highly skewed distributions (skewness > 2)",
        "Data with extreme outliers",
        "Heavy-tailed distributions",
        "When balanced sample sizes per bin are critical",
        "Ordinal encoding for categorical features",
        "Reducing impact of outliers on model training"
    ],
    
    "Consider Alternatives When": [
        "Data is approximately normal",
        "Domain-specific bin boundaries are required",
        "Interpretability of bin ranges is crucial",
        "Many duplicate values exist",
        "Real-time scoring with changing data distributions"
    ],
    
    "Parameter Selection Guidelines": [
        "Start with 3-5 bins for initial exploration",
        "Use 5-10 bins for most practical applications",
        "Consider 10+ bins only for large datasets (>10k samples)",
        "Fewer bins for small datasets (<1k samples)",
        "Test multiple bin counts via cross-validation"
    ]
}

for category, recommendations in decision_framework.items():
    print(f"\n🎯 {category}:")
    for rec in recommendations:
        print(f"   • {rec}")

# Practical workflow recommendation
print(f"\n🔄 Recommended Workflow:")
workflow_steps = [
    "1. Exploratory Data Analysis",
    "   - Check distribution shape (skewness, kurtosis)",
    "   - Identify outliers and extreme values",
    "   - Assess unique value counts",
    
    "2. Method Selection",
    "   - EqualFrequency for skewed data",
    "   - EqualWidth for uniform/normal data",
    "   - Manual binning for domain knowledge",
    
    "3. Parameter Tuning",
    "   - Start with default bin count (5)",
    "   - Test 3, 5, 10 bins via cross-validation",
    "   - Consider data size constraints",
    
    "4. Validation and Testing",
    "   - Check bin frequency balance",
    "   - Validate on holdout data",
    "   - Test edge cases (outliers, duplicates)",
    
    "5. Production Considerations",
    "   - Save fitted binning models",
    "   - Monitor data distribution drift",
    "   - Plan for new data edge cases"
]

for step in workflow_steps:
    print(f"   {step}")

# Create comprehensive comparison matrix
print(f"\n📊 Method Comparison Matrix:")

comparison_matrix = pd.DataFrame({
    'Characteristic': [
        'Skewed Data Handling',
        'Outlier Robustness', 
        'Interpretability',
        'Computational Speed',
        'Memory Efficiency',
        'Parameter Sensitivity',
        'Domain Flexibility',
        'Statistical Balance'
    ],
    'EqualFrequency': [
        'Excellent ⭐⭐⭐',
        'Excellent ⭐⭐⭐',
        'Moderate ⭐⭐',
        'Good ⭐⭐⭐',
        'Good ⭐⭐⭐',
        'Low ⭐⭐⭐',
        'Moderate ⭐⭐',
        'Excellent ⭐⭐⭐'
    ],
    'EqualWidth': [
        'Poor ⭐',
        'Poor ⭐',
        'Excellent ⭐⭐⭐',
        'Excellent ⭐⭐⭐',
        'Excellent ⭐⭐⭐',
        'Low ⭐⭐⭐',
        'High ⭐⭐⭐',
        'Poor ⭐'
    ],
    'Manual': [
        'Variable ⭐⭐',
        'Variable ⭐⭐',
        'Excellent ⭐⭐⭐',
        'Excellent ⭐⭐⭐',
        'Excellent ⭐⭐⭐',
        'Variable ⭐⭐',
        'Excellent ⭐⭐⭐',
        'Variable ⭐⭐'
    ]
})

print(comparison_matrix.to_string(index=False))

# Common pitfalls and solutions
print(f"\n⚠️ Common Pitfalls and Solutions:")

pitfalls = {
    "Pitfall": [
        "Using too many bins for small datasets",
        "Ignoring duplicate values",
        "Not handling missing values",
        "Applying to already discretized data",
        "Not saving fitted models",
        "Ignoring data distribution changes"
    ],
    "Solution": [
        "Use n_bins ≤ sqrt(sample_size)",
        "Check unique value counts first",
        "Impute or drop NaN before binning", 
        "Apply only to continuous features",
        "Use joblib/pickle for model persistence",
        "Monitor and retrain periodically"
    ],
    "Prevention": [
        "Data size analysis",
        "Exploratory data analysis",
        "Preprocessing pipeline",
        "Feature type validation",
        "MLOps best practices",
        "Data drift monitoring"
    ]
}

pitfall_df = pd.DataFrame(pitfalls)
print(pitfall_df.to_string(index=False))

# Performance optimization tips
print(f"\n⚡ Performance Optimization Tips:")

optimization_tips = [
    "• Use vectorized operations when possible",
    "• Consider parallel processing for multiple features",
    "• Cache fitted models for repeated use",
    "• Batch process large datasets",
    "• Use appropriate data types (float32 vs float64)",
    "• Profile memory usage for very large datasets",
    "• Consider approximate quantiles for massive data"
]

for tip in optimization_tips:
    print(f"   {tip}")

# Integration best practices
print(f"\n🔗 Integration Best Practices:")

integration_practices = {
    "Pipeline Design": [
        "Place binning before scaling/normalization",
        "Use ColumnTransformer for mixed data types",
        "Include binning in cross-validation",
        "Test pipeline end-to-end"
    ],
    
    "Model Training": [
        "Compare binned vs unbinned features",
        "Use binning to handle outliers",
        "Consider binning for regularization",
        "Validate on representative data"
    ],
    
    "Production Deployment": [
        "Version control binning models",
        "Monitor input data distribution",
        "Handle new data edge cases",
        "Document bin interpretation"
    ]
}

for category, practices in integration_practices.items():
    print(f"\n   {category}:")
    for practice in practices:
        print(f"      • {practice}")

# Create decision tree visualization
print(f"\n🌳 Decision Tree for Binning Method Selection:")

decision_tree = """
Data Distribution Analysis
├── Highly Skewed (|skewness| > 2)
│   ├── With Outliers → EqualFrequencyBinning ⭐
│   └── Without Outliers → EqualFrequencyBinning ⭐
├── Moderately Skewed (1 < |skewness| ≤ 2)
│   ├── Domain Knowledge Available → Manual Binning
│   └── No Domain Knowledge → EqualFrequencyBinning
├── Approximately Normal (|skewness| ≤ 1)
│   ├── Interpretability Critical → EqualWidthBinning
│   └── Statistical Balance Critical → EqualFrequencyBinning
└── Unknown Distribution
    └── Start with EqualFrequencyBinning (robust default)
"""

print(decision_tree)

# Final recommendation summary
print(f"\n🎯 Executive Summary:")
summary_points = [
    "EqualFrequencyBinning is the robust choice for skewed data",
    "Provides statistical balance across bins",
    "Handles outliers naturally and effectively", 
    "Integrates seamlessly with scikit-learn pipelines",
    "Start with 5 bins and tune via cross-validation",
    "Monitor data distribution changes in production",
    "Combine with domain knowledge when available"
]

for i, point in enumerate(summary_points, 1):
    print(f"   {i}. {point}")

print(f"\n✅ Ready to implement EqualFrequencyBinning in your workflow!")
print(f"   Remember: The best binning method depends on your specific data and use case.")

## 🎯 Conclusion

This comprehensive exploration of EqualFrequencyBinning demonstrates its effectiveness as a robust data preprocessing technique, particularly for handling skewed distributions and outlier-heavy datasets. The method's ability to maintain statistical balance while adapting to data characteristics makes it a valuable tool in the data scientist's toolkit.

In [ ]:
# Final conclusion and key takeaways
print("🎯 EqualFrequencyBinning: Comprehensive Analysis Conclusion")
print("=" * 60)

print("📊 What We've Learned:")

key_findings = {
    "Core Strengths": [
        "✅ Excellent handling of skewed distributions",
        "✅ Natural robustness to outliers and extreme values",
        "✅ Maintains statistical balance across bins", 
        "✅ Seamless scikit-learn pipeline integration",
        "✅ Competitive computational performance",
        "✅ Handles most edge cases gracefully"
    ],
    
    "Key Applications": [
        "🎯 Financial data with heavy-tailed distributions",
        "🎯 Sensor data with outliers and measurement errors",
        "🎯 Web analytics with power-law user behavior",
        "🎯 Preprocessing for algorithms sensitive to outliers",
        "🎯 Feature engineering for tree-based models",
        "🎯 Creating balanced categorical representations"
    ],
    
    "Technical Insights": [
        "🔬 Per-column vs joint fitting modes offer flexibility",
        "🔬 Bin count selection impacts granularity vs stability",
        "🔬 Memory overhead is reasonable for most applications",
        "🔬 Performance scales linearly with data size and features",
        "🔬 Edge cases require minimal special handling",
        "🔬 Cross-validation helps optimize bin count selection"
    ]
}

for category, points in key_findings.items():
    print(f"\n{category}:")
    for point in points:
        print(f"   {point}")

# When to choose EqualFrequencyBinning
print(f"\n🎯 Choose EqualFrequencyBinning When:")
choose_when = [
    "Your data exhibits significant skewness (|skewness| > 1)",
    "Outliers are present and difficult to remove",
    "You need balanced sample sizes across bins",
    "Statistical stability is more important than interpretability",
    "Working with heavy-tailed or power-law distributions",
    "Building models sensitive to feature distributions"
]

for i, condition in enumerate(choose_when, 1):
    print(f"   {i}. {condition}")

# When to consider alternatives
print(f"\n🤔 Consider Alternatives When:")
consider_alternatives = [
    "Data is approximately normally distributed",
    "Domain-specific bin boundaries are required",
    "Interpretability of bin ranges is critical",
    "Real-time applications need consistent boundaries",
    "Many duplicate values prevent equal frequencies"
]

for i, condition in enumerate(consider_alternatives, 1):
    print(f"   {i}. {condition}")

# Implementation checklist
print(f"\n✅ Implementation Checklist:")
checklist = [
    "Analyze data distribution characteristics",
    "Handle missing values before binning",
    "Choose appropriate bin count (typically 3-10)",
    "Validate with cross-validation",
    "Test edge cases and robustness",
    "Monitor performance in production",
    "Document binning strategy and rationale"
]

for i, item in enumerate(checklist, 1):
    print(f"   {i}. {item}")

# Future considerations
print(f"\n🔮 Future Considerations:")
future_items = [
    "Monitor data distribution drift over time",
    "Consider adaptive binning for streaming data",
    "Experiment with hybrid binning approaches",
    "Evaluate impact on model interpretability",
    "Explore integration with automated ML pipelines"
]

for item in future_items:
    print(f"   • {item}")

# Final summary statistics from our analysis
print(f"\n📈 Analysis Summary Statistics:")
summary_stats = {
    "Datasets Analyzed": "4 (skewed, outlier-heavy, mixed, classification)",
    "Bin Configurations Tested": "Multiple (3, 5, 10, 20 bins)",
    "Performance Benchmarks": "Data size, feature count, bin count scaling",
    "Edge Cases Evaluated": "6 (duplicates, small data, outliers, constants, NaN, limited unique)",
    "Pipeline Integrations": "5 (no binning, EqualFreq 3/5/10, EqualWidth 5)",
    "Cross-validation Folds": "5-fold for robust evaluation"
}

for metric, value in summary_stats.items():
    print(f"   {metric}: {value}")

print(f"\n🌟 Final Recommendation:")
print("   EqualFrequencyBinning is a robust, versatile preprocessing tool")
print("   that excels with skewed data and provides statistical balance.")
print("   It should be a primary consideration for data preprocessing")
print("   pipelines, especially when dealing with real-world messy data.")

print(f"\n🚀 Next Steps:")
next_steps = [
    "Apply to your specific dataset",
    "Compare with other binning methods",
    "Integrate into your ML pipeline",
    "Monitor and validate in production",
    "Share insights with your team"
]

for i, step in enumerate(next_steps, 1):
    print(f"   {i}. {step}")

print(f"\n" + "="*60)
print("📝 Thank you for exploring EqualFrequencyBinning!")
print("   This analysis provides a comprehensive foundation for")
print("   effective use in your data science projects.")
print("="*60)

## 3. EqualFrequency vs EqualWidth Comparison

Let's directly compare EqualFrequencyBinning with EqualWidthBinning to understand the key differences and advantages.

In [ ]:
print("⚖️ EqualFrequency vs EqualWidth Comparison")
print("=" * 50)

# Import EqualWidthBinning for comparison
from binlearn.methods import EqualWidthBinning

# Use the highly skewed data for comparison
test_features = ['exponential', 'lognormal']
comparison_data = data_skewed[test_features].copy()

print(f"📊 Testing on highly skewed features: {test_features}")
print(f"Data shape: {comparison_data.shape}")

# Compare both methods
n_bins = 5
comparison_results = {}

methods = {
    'EqualFrequency': EqualFrequencyBinning(n_bins=n_bins),
    'EqualWidth': EqualWidthBinning(n_bins=n_bins)
}

for method_name, binner in methods.items():
    print(f"\n🔍 {method_name} Analysis:")
    
    # Fit and transform
    binner.fit(comparison_data)
    transformed = binner.transform(comparison_data)
    
    # Store results
    comparison_results[method_name] = {
        'binner': binner,
        'transformed': transformed,
        'bin_edges': binner.bin_edges_
    }
    
    # Analyze bin populations for each feature
    for i, feature in enumerate(test_features):
        binned_values = transformed[:, i]
        unique_bins, counts = np.unique(binned_values, return_counts=True)
        
        print(f"  📈 {feature}:")
        print(f"     Bin edges: {np.round(binner.bin_edges_[feature], 3)}")
        print(f"     Bin sizes: {counts} (total: {sum(counts)})")
        print(f"     Size std:  {np.std(counts):.2f} (lower = more balanced)")

# Visualize the comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

for row, feature in enumerate(test_features):
    feature_data = comparison_data[feature].values
    
    # Original distribution
    ax = axes[row, 0]
    ax.hist(feature_data, bins=30, alpha=0.7, color='lightgray', edgecolor='black')
    ax.set_title(f'Original {feature}\n(Skewness: {stats.skew(feature_data):.2f})')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)
    
    # Compare binning methods
    for col, (method_name, result) in enumerate(comparison_results.items(), 1):
        ax = axes[row, col]
        
        # Get binned values for this feature
        feature_idx = list(comparison_data.columns).index(feature)
        binned_values = result['transformed'][:, feature_idx]
        bin_edges = result['bin_edges'][feature]
        
        # Create histogram with bin boundaries
        ax.hist(feature_data, bins=30, alpha=0.6, color='lightblue', 
               edgecolor='black', label='Original')
        
        # Show bin boundaries
        colors = ['red', 'blue']
        for edge in bin_edges:
            ax.axvline(edge, linestyle='--', alpha=0.8, 
                      color=colors[col-1], linewidth=2, 
                      label='Bin edge' if edge == bin_edges[0] else '')
        
        # Calculate and show bin populations
        bin_counts = []
        for i in range(len(bin_edges)-1):
            mask = (feature_data >= bin_edges[i]) & (feature_data < bin_edges[i+1])
            bin_counts.append(np.sum(mask))
        # Handle last bin (include upper boundary)
        mask = feature_data >= bin_edges[-2]
        bin_counts[-1] = np.sum(mask)
        
        ax.set_title(f'{method_name} - {feature}\\nBin sizes: {bin_counts}\\nStd: {np.std(bin_counts):.1f}')
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Quantitative comparison
print(f"\n📊 Quantitative Comparison Summary")
print("=" * 60)
print(f"{'Method':<15} {'Feature':<12} {'Bin Balance':<12} {'Edge Range':<15} {'Performance':<12}")
print("-" * 60)

for method_name, result in comparison_results.items():
    for feature in test_features:
        # Calculate bin balance (lower std = more balanced)
        feature_idx = list(comparison_data.columns).index(feature)
        binned_values = result['transformed'][:, feature_idx]
        unique_bins, counts = np.unique(binned_values, return_counts=True)
        balance_score = np.std(counts)
        
        # Calculate edge range
        edges = result['bin_edges'][feature]
        edge_range = edges[-1] - edges[0]
        
        # Performance indicator
        performance = "🏆 Better" if method_name == "EqualFrequency" and balance_score < 10 else "⚠️ Worse" if balance_score > 50 else "✅ Good"
        
        print(f"{method_name:<15} {feature:<12} {balance_score:<12.1f} {edge_range:<15.2f} {performance:<12}")

print(f"\n🔍 Key Insights:")
print(f"   📊 EqualFrequency creates more balanced bin populations")
print(f"   📏 EqualWidth creates uniform bin widths but uneven populations")
print(f"   🎯 For skewed data, EqualFrequency prevents empty or overcrowded bins")
print(f"   ⚖️ Bin balance measured by standard deviation of bin sizes (lower = better)")
print(f"   🏆 EqualFrequency typically achieves balance_score < 10 for most data")

## 3. Basic Equal Frequency Binning

Let's start with the fundamental usage of `EqualFrequencyBinning` using the fit/transform pattern. Equal frequency binning ensures each bin contains approximately the same number of samples:

In [3]:
# Create a basic EqualFrequencyBinning instance
print("🔧 Creating EqualFrequencyBinning instance...")
basic_binner = EqualFrequencyBinning(n_bins=5, preserve_dataframe=True)

# Fit the binner on DataFrame data (skewed distributions)
print("\n🎯 Fitting binner on DataFrame with skewed data...")
basic_binner.fit(df_data)

# Check the fitted parameters - quantile-based edges
print("\n📋 Fitted parameters (quantile-based bin edges):")
print(f"Number of features: {len(basic_binner.bin_edges_)}")
for feature, edges in basic_binner.bin_edges_.items():
    print(f"  {feature}: {len(edges)-1} bins")
    print(f"    Edges: {np.round(edges, 2)}")
    
    # Show quantiles used
    quantiles = np.linspace(0, 1, len(edges))
    print(f"    Quantiles: {np.round(quantiles, 3)}")

# Transform the data
print("\n🔄 Transforming data...")
df_binned = basic_binner.transform(df_data)

print(f"Original DataFrame shape: {df_data.shape}")
print(f"Binned DataFrame shape: {df_binned.shape}")
print(f"Original data type: {type(df_data)}")
print(f"Binned data type: {type(df_binned)}")

# Show sample transformations
print("\n🔍 Sample transformations:")
for i in range(3):
    print(f"Row {i}:")
    for col in df_data.columns:
        original = df_data.iloc[i][col]
        binned = df_binned.iloc[i][col]
        print(f"  {col}: {original:.2f} → bin {binned}")
    print()

🔧 Creating EqualFrequencyBinning instance...

🎯 Fitting binner on DataFrame with skewed data...

📋 Fitted parameters (quantile-based bin edges):
Number of features: 4
  age: 5 bins
    Edges: [18.44 22.61 25.78 28.15 33.9  55.01]
    Quantiles: [0.  0.2 0.4 0.6 0.8 1. ]
  income: 5 bins
    Edges: [  2395.03  11273.31  17268.1   27493.25  44045.55 138735.99]
    Quantiles: [0.  0.2 0.4 0.6 0.8 1. ]
  score: 5 bins
    Edges: [ 0.72 13.84 23.06 30.29 42.03 73.81]
    Quantiles: [0.  0.2 0.4 0.6 0.8 1. ]
  wait_time: 5 bins
    Edges: [ 0.    0.78  1.54  2.97  4.6  12.57]
    Quantiles: [0.  0.2 0.4 0.6 0.8 1. ]

🔄 Transforming data...
Original DataFrame shape: (200, 4)
Binned DataFrame shape: (200, 4)
Original data type: <class 'pandas.core.frame.DataFrame'>
Binned data type: <class 'pandas.core.frame.DataFrame'>

🔍 Sample transformations:
Row 0:
  age: 31.21 → bin 3
  income: 4535.21 → bin 0
  score: 24.40 → bin 2
  wait_time: 6.69 → bin 4

Row 1:
  age: 38.19 → bin 4
  income: 27629.5